# notion to todoist development notebook

In [292]:
import os
import notion
from notion.client import NotionClient
from todoist.api import TodoistAPI

import tools
from bidict import bidict
from datetime import datetime, timezone

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
todoist_token = "ab9df4df62681fd519648c413750c611cad02048"
notion_token = "a2ae36c266053a72fc564b418a1f578d736228f614ab5925be62f22383894f550dd53004bb48005e699e101a94e00008d23fd6eff24286a48f62f943d3e427ddb14092ffcf94df25e58dd939bfc8"
notion_settings = "https://www.notion.so/jarredgreen/8160da398cf544819c41897865dea6b4?v=8176a9e310ac48bc8f514a068415a431"

In [80]:
man = tools.syncManager(
    todoist_token=todoist_token,
    notion_token=notion_token,
    notion_settings_url=notion_settings
)

### time zone rules

In [62]:
x = man.tasks.collection.get_rows(search="Work on GRB")[0]

In [84]:
p = x.last_edited

In [91]:
p.replace(tzinfo=timezone.utc).astimezone(tz=None)

datetime.datetime(2020, 10, 2, 11, 37, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200), 'CEST'))

In [282]:
row = man.tasks.collection.get_rows(search="Add task sync from notion to todoist")[0]

In [165]:
item = tools.taskManager.from_notion(man, row)

In [307]:
datetime.strptime(item.due, "%Y-%m-%d").date()

datetime.date(2020, 10, 8)

In [322]:
tools.labelManager.get_notion_labels(man,man.tasks.collection.get_rows(search="Add task sync from notion to todoist")[0])

(['Todo⚙️', 'email', 'quick'], [2155295024, 2155305730, 2155305731])

In [335]:
for row in man.labels.collection.get_rows():
    print(row.title, row.relevant_columns)

Urgent! 🔥 status
Doing 🔭 status
To Do ⚙️ status
Waiting ⏳ status
Someday 💤 status
Done 🙌 status
email tags
quick tags


In [348]:
man.tasks.collection.

In [340]:
row.relevant_columns

'tags'

In [141]:
tools.taskManager.to_todoist(manager=man, item=item)

(4227901993, 2690082666)

In [178]:
man.sync_todoist_api()

In [180]:
itemt = tools.taskManager.from_todoist(man, man.api.items.get_by_id(4227901993))

In [193]:
dict(a=5).update(a=4)

In [246]:
def strike(text):
    result = ''
    for c in str(text):
        result = result + c + '\u0336'
    return result

def compare_two_tasks(manager, task1, task2):
    """task 1 takes priority over task 2 if there are differences,
       usually task1 is notion version and task2 is the todoist version."""
    
    task1_dict = task1.__dict__
    task2_dict = task2.__dict__
    
    new_task_dict = task1_dict.copy()
    updated_properties = []
    
    for (key, value1), value2 in zip(task1_dict.items(), task2_dict.values()):
        if value1 != value2 and key != "source":  # exclude the "source" item
            updated_properties.append(key)
            
            if value1 is None:
                print(f"{key}: {strike(task1_dict['source'] + ': ' + str(value1))} -> {task2_dict['source'] + ': ' + str(value2)}")
                new_task_dict[key] = value2
            else:
                print(f"{key}: {strike(task2_dict['source'] + ': ' + str(value2))} -> {task1_dict['source'] + ': ' + str(value1)}")
                new_task_dict[key] = value1
        else:
            new_task_dict[key] = value1
            
    new_task = tools.task(source="merged",
                          task_id = new_task_dict["task_id"],
                          notion_task_id = new_task_dict["notion_task_id"],
                          content=new_task_dict["content"],
                          done=new_task_dict["done"],
                          due=new_task_dict["due"],
                          label_ids=new_task_dict["label_ids"],
                          label_names=new_task_dict["label_names"],
                          project_id=new_task_dict["project_id"],
                          notion_project_id=new_task_dict["notion_project_id"],
                          project_name=new_task_dict["project_name"],
                          todoist_note_id=new_task_dict["todoist_note_id"],
            )
    
    return new_task, updated_properties
            
    

In [248]:
item.__dict__.keys()

dict_keys(['source', 'task_id', 'content', 'done', 'due', 'label_ids', 'label_names', 'project_id', 'project_name', 'notion_task_id', 'notion_project_id', 'todoist_note_id'])

In [247]:
compare_two_tasks(man, item, itemt)

task_id: n̶o̶t̶i̶o̶n̶:̶ ̶N̶o̶n̶e̶ -> todoist: 4227901993
label_ids: t̶o̶d̶o̶i̶s̶t̶:̶ ̶[̶2̶1̶5̶5̶2̶9̶5̶0̶2̶4̶,̶ ̶2̶1̶5̶5̶3̶0̶5̶7̶3̶0̶]̶ -> notion: [2155295024, 2155305730, 2155305731]
label_names: t̶o̶d̶o̶i̶s̶t̶:̶ ̶[̶'̶T̶o̶d̶o̶⚙̶️̶'̶,̶ ̶'̶e̶m̶a̶i̶l̶'̶]̶ -> notion: ['Todo⚙️', 'email', 'quick']
todoist_note_id: n̶o̶t̶i̶o̶n̶:̶ ̶N̶o̶n̶e̶ -> todoist: 2690082666


(Task: Add task sync from notion to todoist
  {
     "source": "merged",
     "task_id": 4227901993,
     "content": "Add task sync from notion to todoist",
     "done": false,
     "due": "2020-10-08",
     "label_ids": [
         2155295024,
         2155305730,
         2155305731
     ],
     "label_names": [
         "Todo\u2699\ufe0f",
         "email",
         "quick"
     ],
     "project_id": 2246178573,
     "project_name": "Notion + Todoist",
     "notion_task_id": "22f46ce5-2ce8-41fa-8c7c-a107b579e25a",
     "notion_project_id": "d35d2707-8e9e-4d1c-8993-1085f2b6435a",
     "todoist_note_id": 2690082666
 },
 ['task_id', 'label_ids', 'label_names', 'todoist_note_id'])

In [244]:
test_item = man.api.items.get_by_id(4227901993)

In [ ]:
test_item.update()